##Geopgraphical Map Try out

In [1]:
import pandas as pd
import numpy as np

#for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt

In [2]:
#saving data into dataframe
df_house = pd.read_csv("data/eda.csv")

In [3]:
# This Python code (OOP approach) creates a map using Altair.

import pandas as pd 
import altair as alt
from vega_datasets import data

def load_data():
    """
    Loads the airports dataset from a CSV file into a Pandas DataFrame

    Returns:
    airports_df (DataFrame): DataFrame containing the airports data
    """
    
    airports_df = pd.read_csv("./data_group_work/airports.csv")
    
    return airports_df


def get_states_data():
    """
    Load the US states GeoJSON data from Altair-Vega dataset

    Returns:
    states (GeoJSON): Altair object containing state geospatial data
    """
    
    states = alt.topo_feature(data.us_10m.url, feature='states')
    
    return states


def create_base_chart(airports_df):
    """
    Creates a base Altair chart showing airport locations.

    Args:
    airports_df (DataFrame): DataFrame containing airport data

    Returns:
    base (Chart): Base chart 
    """

    base = alt.Chart(airports_df).mark_circle().encode(
        longitude='longitude:Q',
        latitude='latitude:Q'
      )
    
    return base


def create_state_layer(states):
    """
    Creates a state background/border layer for the chart.

    Args:
    states (GeoJSON): Altair object containing state geospatial data

    Returns:
    state_layer (Chart): State border layer
    """
    
    state_layer = alt.Chart(states).mark_geoshape().encode(
        fill=alt.value('lightgray'),
        stroke=alt.value('white')
    ).properties(
        width=800,
        height=600,
        title={
            "text": ["US Airports"],
        }
    ).project(
        type='albersUsa'
    )
    
    return state_layer


def create_points_layer(base, selector):
    """
    Creates a points layer, encoding *size and color based on data fields.
        *Data point size based on the relation between IATA code and airports size

    Args:
    base (Chart): Base chart
    Set the color scale to use the 'category20b' scheme
    This is a colorblind-friendly qualitative palette
    selector (Selection): Interval selection

    Returns:
    points (Chart): Points layer
    """
    
    points = base.encode(
        color=alt.Color('state:N', scale=alt.Scale(scheme='category20b'), legend=alt.Legend(title='State')),
        #size=alt.Size('iata:N', legend=alt.Legend(title='IATA Code')),
        opacity=alt.condition(selector, alt.value(0.6), alt.value(0.2)),
        tooltip=['iata:N', 'name:N', 'city:N', 'state:N', 'longitude:Q', 'latitude:Q']
    ).add_params(selector).interactive()
    
    return points

def build_chart(state_layer, points):
    """
    Builds an interactive map of US airports with state and points layers.
    The map shows the location, state, and IATA code of each airport.
    The user can select a region on the map to highlight the airports in that area.
    The map has an inner and outer border frame and a title.

    Args:
    state_layer (Chart): State border layer 
    points (Chart): Points layer

    Returns:
    chart (Chart): An Altair chart object that can be displayed or saved.
    """
    
    chart = alt.layer(
        state_layer,
        points
    ).configure_view(
        stroke='black',
        strokeWidth=1
    ).properties(
        width=1360,
        height=768,
        title={
            "text": ["US Airports"]
        }
    ).project(
        type='albersUsa'
    ).configure( # Add outer border frame
        background='aliceblue',
        padding=10
    )
    
    return chart


def main():
    """
    Loads the data, creates the chart, and displays/saves the output.
    Create a selection interval for interactive highlighting.
    """
    
    # Load data
    airports_df = load_data()
    states = get_states_data()
    
    # Create chart layers
    base = create_base_chart(airports_df)
    selector = alt.selection_interval(empty=True) 
    state_layer = create_state_layer(states)
    points = create_points_layer(base, selector)

    # Build and display chart
    chart = build_chart(state_layer, points)
    chart.display()
    
    # Save output
    chart.save('chart.html')


if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'vega_datasets'